In [2]:
import pymysql #데이터베이스 연결
import numpy as np #수치 계산
from scipy.spatial.distance import euclidean, cosine #거리 계산

In [3]:
# user_record와 album 테이블에서 필요한 데이터를 가져오는 함수
def fetch_records_and_albums(connection):
    with connection.cursor() as cursor: #데이터베이스 커서가 생성되고, 블록이 끝나면 자동으로 닫힘
        cursor.execute("SELECT user_id, album_id FROM user_record") # user_record 테이블에서 user_id와 album_id를 가져오기 위한 SQL 쿼리 실행
        records = cursor.fetchall()  # 쿼리 결과를 records 변수에 저장
        
        cursor.execute("""
            SELECT id, music_acousticness, music_danceability, music_energy, 
                   music_liveness, music_loudness, music_speechiness, 
                   music_tempo, music_valence 
            FROM album
        """) # album 테이블에서 id와 여러 특성 값을 가져오기 위한 SQL 쿼리 실행
        albums = cursor.fetchall()  # 쿼리 결과를 albums 변수에 저장
    return records, albums  # 가져온 데이터를 반환

In [5]:
def main():
    connection = pymysql.connect(
        host='mitidb.cvm64ss6y2xv.ap-northeast-2.rds.amazonaws.com',       
        user='minseo',  
        password='Alstj!!809', 
        database='mitiDB'
    ) # MySQL 연결

    try:
        records, albums = fetch_records_and_albums(connection) # user_record, album 데이터 가져옴

        album_dict = {album[0]: np.array(album[1:]) for album in albums} # 앨범의 id를 키로 하고, 특성 값을 값으로 하는 딕셔너리를 생성

        for user_id, album_id in records: # 각 사용자가 청취한 앨범에 대해 반복
            current_features = album_dict[album_id]  # 현재 앨범의 특성 값

            closest_album_id_euclidean = None  # 유클리드 거리 기준 가장 가까운 앨범 id 초기화
            closest_distance_euclidean = float('inf')  # 초기 가장 작은 유클리드 거리 설정

            closest_album_id_cosine = None  # 코사인 유사도 기준 가장 가까운 앨범 id 초기화
            closest_distance_cosine = float('inf')  # 초기 가장 작은 코사인 거리 설정
            
            for other_album_id, features in album_dict.items(): # 나머지 앨범들과 현재 앨범의 유사도 계산
                if other_album_id != album_id:  # 동일한 앨범은 비교하지 않음
                    # 유클리드 거리 계산
                    euclidean_distance = euclidean(current_features, features)
                    if euclidean_distance < closest_distance_euclidean:
                        closest_distance_euclidean = euclidean_distance
                        closest_album_id_euclidean = other_album_id

                    # 코사인 유사도 계산
                    cosine_distance = cosine(current_features, features)
                    if cosine_distance < closest_distance_cosine:
                        closest_distance_cosine = cosine_distance
                        closest_album_id_cosine = other_album_id

            # 결과 출력
            print(f"current_album_id: {album_id}, Closest album by Cosine similarity: {closest_album_id_cosine}")


    except pymysql.MySQLError as e:
        print(f"Error: {e}")

    finally:
        connection.close()  # 데이터베이스 연결 닫기

if __name__ == "__main__":
    main()

current_album_id: 1, Closest album by Cosine similarity: 15
current_album_id: 6, Closest album by Cosine similarity: 17
current_album_id: 29, Closest album by Cosine similarity: 5


record에 있는 각 앨범마다 유사한 값을 비교하는게 아닌 record에 담긴 앨범 특징의 평균값과 유사한 곡들을 추출하는 것으로 변경

In [9]:
import pymysql
import numpy as np
from scipy.spatial.distance import euclidean, cosine

# user_record와 album 테이블에서 필요한 데이터를 가져오는 함수
def fetch_records_and_albums(connection):
    with connection.cursor() as cursor:
        # user_record 테이블에서 user_id와 album_id를 가져오기 위한 SQL 쿼리 실행
        cursor.execute("SELECT user_id, album_id FROM user_record")
        records = cursor.fetchall()  # 쿼리 결과를 records 변수에 저장
        
        # album 테이블에서 id와 여러 특성 값을 가져오기 위한 SQL 쿼리 실행
        cursor.execute("""
            SELECT id, music_acousticness, music_danceability, music_energy, 
                   music_liveness, music_loudness, music_speechiness, 
                   music_tempo, music_valence 
            FROM album
        """)
        albums = cursor.fetchall()  # 쿼리 결과를 albums 변수에 저장
    return records, albums  # 가져온 데이터를 반환


def calculate_average_features(user_records, album_dict): # user_records에 포함된 album_id에 대해 평균 특성 벡터를 계산
    feature_sums = np.zeros(len(next(iter(album_dict.values()))))  # 첫 번째 앨범의 특성 벡터 길이만큼 0으로 초기화
    count = 0

    for user_id, album_id in user_records:
        feature_sums += album_dict[album_id]
        count += 1

    if count == 0:
        return feature_sums
    else:
        return feature_sums / count  # 평균 특성 벡터 계산


# 메인 함수
def main():
    # MySQL 데이터베이스에 연결 설정
    connection = pymysql.connect(
        host='mitidb.cvm64ss6y2xv.ap-northeast-2.rds.amazonaws.com',       
        user='minseo',  
        password='Alstj!!809', 
        database='mitiDB'
    )

    try:
        records, albums = fetch_records_and_albums(connection) # 데이터베이스에서 레코드와 앨범 데이터를 가져옴

        album_dict = {album[0]: np.array(album[1:]) for album in albums} # 앨범의 id를 키로 하고, 특성 값을 값으로 하는 딕셔너리를 생성

        average_features = calculate_average_features(records, album_dict) # user_record에 있는 모든 user_id에 대해 평균 특성 벡터 계산

        similarities = []
        for album_id, features in album_dict.items():
            # 유클리드 거리 계산
            euclidean_distance = euclidean(average_features, features)
            similarities.append((album_id, euclidean_distance))

        top_20_albums = sorted(similarities, key=lambda x: x[1])[:20] # 유사도 기준으로 상위 20개의 앨범 선택

        # 결과 출력
        print("< Top 20 Albums >")
        for album_id, distance in top_20_albums:
            print(f"Album ID: {album_id}")
    
    except pymysql.MySQLError as e:
        print(f"Error: {e}")

    finally:
        connection.close()  # 데이터베이스 연결 닫기

if __name__ == "__main__":
    main()


< Top 20 Albums >
Album ID: 75
Album ID: 155
Album ID: 7
Album ID: 18
Album ID: 31
Album ID: 92
Album ID: 102
Album ID: 125
Album ID: 135
Album ID: 145
Album ID: 179
Album ID: 153
Album ID: 122
Album ID: 111
Album ID: 119
Album ID: 63
Album ID: 159
Album ID: 8
Album ID: 19
Album ID: 32


In [12]:
import pymysql
import numpy as np
from scipy.spatial.distance import euclidean

# user_record와 album 테이블에서 필요한 데이터를 가져오는 함수
def fetch_records_and_albums(connection):
    with connection.cursor() as cursor:
        # user_record 테이블에서 user_id와 album_id 가져오기
        cursor.execute("SELECT user_id, album_id FROM user_record")
        records = cursor.fetchall()
        
        # album 테이블에서 id와 여러 특성 값 가져오기
        cursor.execute("""
            SELECT id, music_acousticness, music_danceability, music_energy, 
                   music_liveness, music_loudness, music_speechiness, 
                   music_tempo, music_valence 
            FROM album
        """)
        albums = cursor.fetchall()
    return records, albums

# user_records에 포함된 album_id에 대해 평균 특성 벡터를 계산하는 함수
def calculate_average_features(user_records, album_dict):
    feature_sums = np.zeros(len(next(iter(album_dict.values()))))  # 첫 번째 앨범의 특성 벡터 길이만큼 0으로 초기화
    count = 0

    for user_id, album_id in user_records:
        feature_sums += album_dict[album_id]
        count += 1

    if count == 0:
        return feature_sums
    else:
        return feature_sums / count  # 평균 특성 벡터 계산

# main 함수
def main():
    # MySQL 데이터베이스에 연결
    connection = pymysql.connect(
        host='mitidb.cvm64ss6y2xv.ap-northeast-2.rds.amazonaws.com',       
        user='minseo',  
        password='Alstj!!809', 
        database='mitiDB'
    )

    try:
        records, albums = fetch_records_and_albums(connection)

        # album_id를 키로 하고, 여러 특성을 값으로 하는 딕셔너리 생성
        album_dict = {album[0]: np.array(album[1:]) for album in albums}

        # user_record에 있는 모든 user_id에 대해 평균 특성 벡터 계산
        average_features = calculate_average_features(records, album_dict)

        similarities = []
        for album_id, features in album_dict.items():
            # 유클리드 거리 계산
            euclidean_distance = euclidean(average_features, features)
            similarities.append((album_id, euclidean_distance))

        # 유사도 기준으로 상위 20개의 앨범 선택
        top_20_albums = sorted(similarities, key=lambda x: x[1])[:20]

        # 결과를 customized_rec 테이블에 삽입
        with connection.cursor() as cursor:
            for user_id, _ in records:  # 모든 user_id에 대해 반복
                for album_id, distance in top_20_albums:
                    try:
                        cursor.execute(
                            "INSERT INTO customized_rec (user_id, album_id) VALUES (%s, %s)",
                            (user_id, album_id)
                        )
                    except pymysql.IntegrityError:
                        # 중복 키 에러가 발생하면 해당 항목을 삽입하지 않음
                        continue
            connection.commit()  # 변경사항을 커밋하여 데이터베이스에 반영

        # 결과 출력
        print("앨범 삽입 완료")

    except pymysql.MySQLError as e:
        print(f"Error: {e}")

    finally:
        connection.close()  # 데이터베이스 연결 닫기

if __name__ == "__main__":
    main()


앨범 삽입 완료


In [ ]:
#최종

import pymysql #mysql 연동
import numpy as np #숫자 배열&계산
from scipy.spatial.distance import euclidean #유클리드 거리 계산

#user_record와 album 테이블에서 데이터 가져오는 함수
def fetch_records_and_albums(connection):
    #db연결할 때 커서를 생성하고, sql쿼리를 실행한 후 블록 끝나면 커서가 자동으로 닫힘
    with connection.cursor() as cursor:
        #user_record 테이블에서 user_id와 album_id 가져옴
        cursor.execute("SELECT user_id, album_id FROM user_record")
        records = cursor.fetchall()
        #album 테이블에서 id와 크롤링한 필드 값 가져옴
        cursor.execute("""
            SELECT id, music_acousticness, music_danceability, music_energy, 
                   music_liveness, music_loudness, music_speechiness, 
                   music_tempo, music_valence 
            FROM album
        """)
        albums = cursor.fetchall()
    #결과값 튜플로 반환
    return records, albums

#사용자가 재생한 음악들의 평균 특성 벡터 계산
#album_dict는 앨범id를 키로 하고, 특성 벡터를 값으로 가지는 딕셔너리
def calculate_average_features(user_records, album_dict):
    #첫 번째 앨범의 특성 벡터 길이만큼 0으로 초기화
    feature_sums = np.zeros(len(next(iter(album_dict.values()))))
    #재생한 앨범 수 기록할 변수 0으로 초기화
    count = 0

    #재생 기록된 음악에서 앨범 아이디에 대한 특성 벡터 가져와 배열에 더함
    for user_id, album_id in user_records:
        feature_sums += album_dict[album_id]
        count += 1

    #사용자가 감상한 앨범이 없는 경우, 그대로 반환
    if count == 0:
        return feature_sums
    else:
        #아닌 경우, 평균 특성 벡터 계산
        return feature_sums / count  

#main 함수
def main():
    #mysql 연결
    connection = pymysql.connect(
        host='mitidb.cvm64ss6y2xv.ap-northeast-2.rds.amazonaws.com',       
        user='minseo',  
        password='Alstj!!809', 
        database='mitiDB'
    )

    try:
        #user_record, album 테이블에서 데이터 가져옴
        records, albums = fetch_records_and_albums(connection)

        #album_id를 키로 하고,앨범 특성을 값으로 하는 딕셔너리 생성
        album_dict = {album[0]: np.array(album[1:]) for album in albums}

        #user_record에 있는 모든 user_id에 대해 평균 특성 벡터 계산
        average_features = calculate_average_features(records, album_dict)

        #각 앨범 특성 벡터와 사용자 평균 특성 벡터 간 유클리드 거리 게산 후 리스트에 저장
        similarities = []
        for album_id, features in album_dict.items():
            #유클리드 거리 계산
            euclidean_distance = euclidean(average_features, features)
            #앨범 id, 유클리드 거리 쌍으로 구성하여 저장
            similarities.append((album_id, euclidean_distance))

        #유사도 기준 상위 20개 앨범 선택
        top_20_albums = sorted(similarities, key=lambda x: x[1])[:20]

        #결과 customized_rec 테이블에 삽입
        with connection.cursor() as cursor:
            #모든 user_id에 대해 반복
            for user_id, _ in records:  
                for album_id, distance in top_20_albums:
                    try:
                        cursor.execute(
                            "INSERT INTO customized_rec (user_id, album_id) VALUES (%s, %s)",
                            (user_id, album_id)
                        )
                    #중복 키 에러가 발생시 해당 항목을 삽입하지 않음
                    except pymysql.IntegrityError:
                        continue
            #변경사항을 커밋하여 데이터베이스에 반영
            connection.commit()  

        #결과 출력
        print("Top 20 albums by Euclidean distance have been inserted into customized_rec table.")

    except pymysql.MySQLError as e:
        print(f"Error: {e}")

    finally:
        #데이터베이스 연결 닫기
        connection.close()  

if __name__ == "__main__":
    main()
